In [1]:
import tensorflow as tf
import numpy as np

Vi starter med at loaded den givne data ind. Her er dataen sepereret i lyd med musik og lyd uden musik. Det skal vi så sætte sammen i et array og lave en array som indeholder om hvilken type hver styk data er. Her kan vi bruge numpy siden hver fil er i .npy format

In [2]:
music_data = np.load("music_data.npy")
other_data = np.load("other_data.npy")
test_data = np.load("test_data.npy")

Data layoutet er her $10500$ 2d arrays af $30\times79$ som repræsentere et log-mel spektogram udregnet fra den givne lyd sample. $x$ dimensionen er frekvens båndene mens $y$ er tids rammen

Nu skal vi sætte `music_data` og `other_data` sammen til et array og lave en array som indeholder hvilken type hver lyd sample er. Vi vil også gerne have noget test data så vi kan evaluere hvor godt netværket er uden at overfitte Det gør vi ved at efterlade 1000 fra `music_dat` og `other_data`

In [3]:
music_train_data = music_data[0: -1000]
other_train_data = other_data[0: -1000]
train_data = (
	np.concatenate((music_train_data, other_train_data)),
	np.concatenate((
		np.full((len(music_train_data)), (1)), 
		np.full((len(other_train_data)), (0))
	))
)	

music_test_data = music_data[len(music_data) - 1000:None]
other_test_data = other_data[len(other_data) - 1000:None]
test_data = (
	np.concatenate((music_test_data, other_test_data)),
	np.concatenate((
		np.full((len(music_test_data)), (1)),
		np.full((len(other_test_data)), (0))
	))
)


Nu har vi lavet dataset til test og træning, nu skal vi sætte vores model op

In [4]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Reshape((30, 79, 1), input_shape = (30, 79)),
	tf.keras.layers.Conv2D(3, (3, 3), activation = "relu"),
	tf.keras.layers.MaxPool2D(),
	tf.keras.layers.Conv2D(32, (3, 3), activation = "relu"),
	tf.keras.layers.MaxPool2D(),
	tf.keras.layers.Conv2D(64, (3, 3), activation = "relu"),
	tf.keras.layers.MaxPool2D(),
	tf.keras.layers.Flatten(),
	tf.keras.layers.Dense(1024, activation = "relu"),
	tf.keras.layers.Dense(512, activation = "relu"),
	tf.keras.layers.Dense(1, activation = "sigmoid")
])

model.build(input_shape=(30, 79))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 30, 79, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 28, 77, 3)         30        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 38, 3)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 36, 32)        896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 18, 32)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 16, 64)         1

Nu hvor vi har en model, så skal den bare trænes

In [5]:
predictions = model(train_data[0][:10]).numpy()
predictions

array([[0.4130155 ],
       [0.36750004],
       [0.44021606],
       [0.44221535],
       [0.39781332],
       [0.341931  ],
       [0.3665561 ],
       [0.42156693],
       [0.37554705],
       [0.40658802]], dtype=float32)

Nu laver vi en loss funktion, siden vi kun har et output så bruger vi binarycrossentropy

In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Så træner vi modellen med dataen over 10 epochs

In [7]:
history = model.fit(train_data[0], train_data[1], epochs=10)

Epoch 1/10
594/594 [==============================] - 16s 26ms/step - loss: 0.2645 - accuracy: 0.8862
Epoch 2/10
594/594 [==============================] - 17s 29ms/step - loss: 0.1400 - accuracy: 0.9461
Epoch 3/10
594/594 [==============================] - 16s 26ms/step - loss: 0.0981 - accuracy: 0.9641
Epoch 4/10
594/594 [==============================] - 17s 28ms/step - loss: 0.0715 - accuracy: 0.9744
Epoch 5/10
594/594 [==============================] - 15s 25ms/step - loss: 0.0536 - accuracy: 0.9792
Epoch 6/10
594/594 [==============================] - 16s 27ms/step - loss: 0.0428 - accuracy: 0.9853
Epoch 7/10
594/594 [==============================] - 16s 27ms/step - loss: 0.0390 - accuracy: 0.9852
Epoch 8/10
594/594 [==============================] - 15s 25ms/step - loss: 0.0302 - accuracy: 0.9891
Epoch 9/10
594/594 [==============================] - 15s 24ms/step - loss: 0.0284 - accuracy: 0.9898
Epoch 10/10
594/594 [==============================] - 15s 25ms/step - loss: 0.033

In [8]:
train_evaluation = model.evaluate(train_data[0], train_data[1])
test_evaluation = model.evaluate(test_data[0], test_data[1])
print("Training set accuracy %.3f%% and loss: %.3f%%" % 
	(train_evaluation[1] * 100, train_evaluation[0] * 100))
print("Test set accuracy %.3f%% and loss: %.3f%%" % 
	(test_evaluation[1] * 100, test_evaluation[0] * 100))

63/63 [==============================] - 1s 8ms/step - loss: 0.0819 - accuracy: 0.9705
Training set accuracy 99.505% and loss: 1.535%
Test set accuracy 97.050% and loss: 8.191%
